In [2]:
from models.DeepQLearning import DDQNAgent, training_loop
from envs.test_env import Electric_Car
import torch
from tqdm import tqdm
import numpy as np
test_env = Electric_Car('data/validate.xlsx')
train_env = Electric_Car('data/train.xlsx')

max_episodes = 250_000
eps_decay = 10_000
eps_start = 1.0
eps_end = .05
discount = .5
learning_rate = 5e-4
buffer = 50_000


model = DDQNAgent('data/train.xlsx', device='cpu', epsilon_decay=eps_decay, epsilon_start=eps_start,
                   epsilon_end=eps_end, discount_rate=discount, lr=learning_rate, buffer_size=buffer)
model.init_envs(train_env)
training_loop(train_env, model, max_episodes)
torch.save(model.target_network.state_dict(), 'model_checkpoints/discount_05.pth')

c:\Users\snipercapt\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Please wait, the experience replay buffer will be filled with random transitions
Initialization with random transitions is done!


<All keys matched successfully>

In [4]:
sums = []
for i in tqdm(range(10)):
    # rews, actions = model.play_game(train_env)
    rews, actions = model.play_game(test_env)
    sums.append(sum(rews))
np.mean(sums)

100%|██████████| 10/10 [00:32<00:00,  3.26s/it]


-656.8800555555557

|CHANGE          |TEST LOSS|   TRAIN LOSS|
|----------|--------|---------|
|basic|         631|         880|
decay/2 |        636|         899|
decay*2  |       638|        880|
discount 0.9|    628 |        876|
discount 0.5 |   656  |       884|
decay/2       |  XXX   |      XXX |